In [5]:
import numpy as np
import cv2
from numba import njit

In [9]:
def constructIndices(edges):
    ''' This function returns the indices of the non zero edges     
    '''
    args = np.argwhere(edges > 0)
    I = args[:, 1]
    J = args[:, 0]
    return I, J

In [10]:
def findMaxima(hSpace, numOfMaxima, delta):
    ''' This function returns the local maximas of the accumulator.
        :param accum: The accumulator for the query image.
        :param delta: The radius around the local maxima to set to zero.
        :rtype: list(tuple)    
    '''
    maxs = []
    
    for i in range(numOfMaxima):
        
        mx = np.argwhere(hSpace == np.max(hSpace))[0]
        y, x = mx[0], mx[1]
        maxs.append((x ,y))
        hSpace[y - delta : y + delta, x - delta : x + delta] = 0
        
    return maxs

In [11]:
@njit
def checkBelongs(I, J, x_c, y_c, R):
    ''' This function find the indices that a point belongs to a circle.
    :param I: int[:] // Indicis of possible circle point 
    :param x_c: int // x coordinates of the circle.
    :param R: int // radius of the circle.
    :rtype: int[:]

    '''
    d = (I - x_c) ** 2 + (J - y_c) ** 2
    return np.where(np.abs(d - R ** 2) < 1e-3, 1, 0)

In [12]:
@njit(cache=True)
def constructAccum(I, J, R, shape):
    ''' This function construct the accumulator for a circle with radius R.
        :param shape: 
    '''
    m, n = shape
    accum = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            accum[i, j] += np.sum(checkBelongs(I, J, j, i, R))
    return accum        

In [13]:
## We fix the radius to 58
img = 255 - cv2.imread('test2.png', 0)

cv2.imshow('Circle', img)
cv2.waitKey()
cv2.destroyAllWindows()

I, J = constructIndices(img)

A = constructAccum(I, J, 58, img.shape)

maxs = findMaxima(A, 2, 5)

for mx in maxs:
    cv2.circle(img, mx, 58, (255, 255, 255), 3)

cv2.imshow('Circle', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
%timeit constructAccum(I, J, 58, img.shape)

1 loop, best of 5: 415 ms per loop
